# Set up 

## Import libraries 

In [1]:
# ALTERED: Removed STANDALONE_TEST feature
# ALTERED: Removed total_cost parameter (cost calculated in FEA_Loop only)
# Parameters - papermill will inject these values
# Tag this cell with "parameters" in the notebook

df_candidates_path = ""
df_crossed_path = ""
df_labeled_path = ""
df_labeled_crossed_path = ""
df_obs_ent_path = ""
df_clause_path = ""
embedding_cache_path = ""

In [2]:
# Parameters
df_candidates_path = "fea_iterations/temp_data/df_candidates.pkl"
df_crossed_path = "fea_iterations/temp_data/df_crossed.pkl"
df_labeled_path = "fea_iterations/temp_data/df_labeled.pkl"
df_labeled_crossed_path = "fea_iterations/temp_data/df_labeled_crossed.pkl"
df_obs_ent_path = "fea_iterations/temp_data/df_obs_ent.pkl"
df_clause_path = "fea_iterations/temp_data/df_clause.pkl"
embedding_cache_path = "fea_iterations/temp_data/embedding_cache.pkl"


In [3]:

import pandas as pd
import pickle
import importlib
import numpy as np
import sys
import os

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"Added current directory to sys.path: {current_dir}")
import free_entailments_algorithm_utils as fea
import scrapbook as sb

print(f"Loading data from pickle files...")
print(f"  Loading from: {df_candidates_path}")

df_candidates = pd.read_pickle(df_candidates_path)
df_crossed = pd.read_pickle(df_crossed_path)
df_labeled = pd.read_pickle(df_labeled_path)
df_labeled_crossed = pd.read_pickle(df_labeled_crossed_path)
df_obs_ent = pd.read_pickle(df_obs_ent_path)
df_clause = pd.read_pickle(df_clause_path)

with open(embedding_cache_path, 'rb') as f:
    embedding_cache_finetuned = pickle.load(f)

print(f"✓ Successfully loaded all data:")
print(f"  - df_candidates: {len(df_candidates)} rows")
print(f"  - df_crossed: {len(df_crossed)} rows")
print(f"  - df_labeled: {len(df_labeled)} rows")
print(f"  - df_labeled_crossed: {len(df_labeled_crossed)} rows")
print(f"  - df_obs_ent: {len(df_obs_ent)} rows")
print(f"  - df_clause: {len(df_clause)} rows")
print(f"  - embedding_cache: {len(embedding_cache_finetuned)} entries")

Added current directory to sys.path: c:\Users\aesteva\Documents\GitHub\fea_project


Loading data from pickle files...
  Loading from: fea_iterations/temp_data/df_candidates.pkl


✓ Successfully loaded all data:
  - df_candidates: 1000000 rows
  - df_crossed: 163 rows
  - df_labeled: 10 rows
  - df_labeled_crossed: 4 rows
  - df_obs_ent: 2 rows
  - df_clause: 63909 rows
  - embedding_cache: 63909 entries


In [4]:
importlib.reload(fea)

<module 'free_entailments_algorithm_utils' from 'c:\\Users\\aesteva\\Documents\\GitHub\\fea_project\\free_entailments_algorithm_utils.py'>

# Calculate Similarity On LLM Results

In [5]:
# OPTIMIZED: Using fine-tuned embeddings from cache (instant!)
# Old way: Re-encoded all texts (~3-5 minutes)
# New way: Lookup from cache (< 1 second)

df_candidates_with_scores = fea.generate_new_bert_results(
    df_candidates,
    text_col1='text1',
    text_col2='text2',
    model_path="./fine_tuned_bi_model",
    new_col="new_cos_sim_score",
    # NEW PARAMETERS: Use the fine-tuned embedding cache!
    embedding_cache=embedding_cache_finetuned,
    id_col1='id1',
    id_col2='id2'
)
df_candidates_with_scores.head()

Using pre-computed embeddings from cache...


c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,verdict,equivalents1,equivalents2,alpha,new_cos_sim_score
0,B0001001p,B0005008p,NaN,[],[],NaN,0.513672
1,B0001001p,B0020004p,NaN,[],[],NaN,0.684570
2,B0001001p,B0083007p,NaN,[],[],NaN,0.668457
3,B0001001p,B0115001p,NaN,[],[],NaN,0.639160
4,B0001001p,B0119005p,NaN,[],[],NaN,0.531738


In [6]:
df_labeled = fea.generate_new_bert_results(
    df_labeled,
    text_col1='text1',
    text_col2='text2',
    model_path="./fine_tuned_bi_model",
    new_col="new_cos_sim_score",
    embedding_cache=embedding_cache_finetuned,
    id_col1='id1',
    id_col2='id2'
)

df_labeled.head()

Using pre-computed embeddings from cache...


c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,text1,text2,verdict,equivalents1,equivalents2,alpha,new_cos_sim_score
0,B0448006p,B1089003p,The emphasis on a singular sovereign power in ...,The accountability of the sovereign to God emp...,NO,[],[],NaN,0.763184
1,B1089003p,B0448006p,The accountability of the sovereign to God emp...,The emphasis on a singular sovereign power in ...,NO,[],[],NaN,0.763184
2,B0674004p,S0051696006p,Maintaining order and justice is essential for...,Maintaining democratic oversight is crucial to...,NO,[],[],NaN,0.738770
3,S0051696006p,B0674004p,Maintaining democratic oversight is crucial to...,Maintaining order and justice is essential for...,NO,[],[],NaN,0.738770
4,B0427001sc,B0596001sc,Popular Estates are essential for a just monar...,Lawful political authority is essential for so...,NO,[],[],NaN,0.765625


In [7]:
## Takes a few minutes depending on computing power


# df_main = fea.add_cross_encoder_score(
#     df_main,                 
#     text_col1='text1',
#     text_col2='text2',
#     model_name="./fine_tuned_nli_model",  
#     new_col="nli_score",
#     batch_size=128              # Keep batch size lower for Cross-Encoders, go easy on your computer
# )

# df_main.head()
# df_main.shape

In [8]:
df_crossed = fea.add_cosine_similarity_from_text(
    df_crossed,
    text_col1="text1",
    text_col2="text2",
    model_name="./fine_tuned_bi_model",  # Ignored when cache provided
    batch_size=128,
    show_progress_bar=False,  # No need since we're using cache
    # NEW PARAMETERS: Use the fine-tuned embedding cache!
    embedding_cache=embedding_cache_finetuned,
    id_col1='id1',
    id_col2='id2'
)

df_crossed.head()

Using pre-computed embeddings from cache...


c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,text1,text2,verdict,cosine_sim
0,B0017003p,B0083004p,The establishment of a governing body or autho...,The legitimacy of governance is rooted in the ...,NaN,0.733887
1,B0078002p,B0083004p,Filmer asserts that there is only one legitima...,The legitimacy of governance is rooted in the ...,NaN,0.585449
2,B0085003p,B0132002p,Blind faith in a ruler is flawed because it di...,The essence of legitimate governance lies in t...,NaN,0.607422
3,B0256001p,B0132002p,Nations can indeed meet and confer sovereignty...,The essence of legitimate governance lies in t...,NaN,0.639648
4,B0293009p,B0132002p,The structure of Parliament minimizes the risk...,The essence of legitimate governance lies in t...,NaN,0.616211


In [9]:
df_labeled_crossed = fea.add_cosine_similarity_from_text(
    df_labeled_crossed,
    text_col1="text1",
    text_col2="text2",
    model_name="./fine_tuned_bi_model",  # Ignored when cache provided
    batch_size=128,
    show_progress_bar=False,  # No need since we're using cache
    # NEW PARAMETERS: Use the fine-tuned embedding cache!
    embedding_cache=embedding_cache_finetuned,
    id_col1='id1',
    id_col2='id2'
)

df_labeled_crossed.head()

Using pre-computed embeddings from cache...


c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,text1,text2,verdict,cosine_sim
0,B0083004p,B0083004p,The legitimacy of governance is rooted in the ...,The legitimacy of governance is rooted in the ...,NaN,1.0
1,B0132002p,B0132002p,The essence of legitimate governance lies in t...,The essence of legitimate governance lies in t...,NaN,1.0
2,B0132002p,B0132002p,The essence of legitimate governance lies in t...,The essence of legitimate governance lies in t...,NaN,1.0
3,B0083004p,B0083004p,The legitimacy of governance is rooted in the ...,The legitimacy of governance is rooted in the ...,NaN,1.0


# Features

## Compute Cos Sim Neighborhood Score

In [10]:
df_candidates = fea.compute_neighbor_weighted_score(
    df5 = df_crossed,
    df6 = df_candidates_with_scores,
    id1_col = "id1",
    id2_col = "id2",
    cosim_df5_col = "cosine_sim",
    cosim_df6_col = "new_cos_sim_score",
    alpha_col = "alpha",
    eq1_col = "equivalents1",
    eq2_col = "equivalents2",
    new_col = "cos_sim_neighbor_score",
)
df_candidates.head()

c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,verdict,equivalents1,equivalents2,alpha,new_cos_sim_score,cos_sim_neighbor_score
0,B0001001p,B0005008p,NaN,[],[],NaN,0.513672,0.513672
1,B0001001p,B0020004p,NaN,[],[],NaN,0.684570,0.684570
2,B0001001p,B0083007p,NaN,[],[],NaN,0.668457,0.668457
3,B0001001p,B0115001p,NaN,[],[],NaN,0.639160,0.639160
4,B0001001p,B0119005p,NaN,[],[],NaN,0.531738,0.531738


In [11]:
df_labeled = fea.compute_neighbor_weighted_score(
    df5 = df_labeled_crossed,
    df6 = df_labeled,
    id1_col = "id1",
    id2_col = "id2",
    cosim_df5_col = "cosine_sim",
    cosim_df6_col = "new_cos_sim_score",
    alpha_col = "alpha",
    eq1_col = "equivalents1",
    eq2_col = "equivalents2",
    new_col = "cos_sim_neighbor_score",
)
df_labeled.head()

c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,text1,text2,verdict,equivalents1,equivalents2,alpha,new_cos_sim_score,cos_sim_neighbor_score
0,B0448006p,B1089003p,The emphasis on a singular sovereign power in ...,The accountability of the sovereign to God emp...,NO,[],[],NaN,0.763184,0.763184
1,B1089003p,B0448006p,The accountability of the sovereign to God emp...,The emphasis on a singular sovereign power in ...,NO,[],[],NaN,0.763184,0.763184
2,B0674004p,S0051696006p,Maintaining order and justice is essential for...,Maintaining democratic oversight is crucial to...,NO,[],[],NaN,0.738770,0.738770
3,S0051696006p,B0674004p,Maintaining democratic oversight is crucial to...,Maintaining order and justice is essential for...,NO,[],[],NaN,0.738770,0.738770
4,B0427001sc,B0596001sc,Popular Estates are essential for a just monar...,Lawful political authority is essential for so...,NO,[],[],NaN,0.765625,0.765625


In [12]:

df_candidates.shape, df_labeled.shape

((1000000, 8), (10, 10))

## Compute NLI Score 

In [13]:
# df_crossed = fea.add_cross_encoder_score(
#     df_crossed,
#     text_col1="text1",
#     text_col2="text2",
    
#     # FIX: Use the relative path with ./ just like before
#     model_name="./fine_tuned_nli_model", 
#     batch_size=128,
#     new_col="nli_score" 
# )

# # (add_cross_encoder_score adds the column in-place)
# if "nli_score" not in df_candidates.columns:
#     print("Scores added to df_crossed!")
    
# df_crossed.head()

# # We reuse the same function used for Cosine Similarity, but point to NLI columns.
# df_candidates = fea.compute_neighbor_weighted_score(
#     df5=df_crossed,
#     df6=df_candidates,
#     id1_col="id1",
#     id2_col="id2",
#     cosim_df5_col="nli_score",    # The 'sigma' lookup table uses NLI
#     cosim_df6_col="nli_score",    # The 'sigma_ij' value uses NLI
#     alpha_col="alpha",
#     eq1_col="equivalents1",
#     eq2_col="equivalents2",
#     new_col="nli_neighbor_score"
# )

# cols_to_show = ['text1', 'text2', 'nli_score', 'nli_neighbor_score', 'verdict']
# df_candidates[cols_to_show].head()

## Compute Transitivity Score

In [14]:
# We calculate graph-based scores (Transitivity).
# 'graph_entailment_score': A -> B (Path Decay applied)
# 'graph_equivalence_score': A <-> B (Stronger constraint)

df_candidates = fea.add_graph_features(
    df=df_candidates,
    entailment_df=df_obs_ent,
    id1_col="id1",
    id2_col="id2",
    verdict_col="verdict",
    positive_label="YES",
    decay=0.9,  # Confidence drops by 10% per extra hop
    max_hops=5
)

print("\nGraph Score Stats:")
print(df_candidates[['graph_entailment_score', 'graph_equivalence_score']].describe())

Building Directed Entailment Graph...
Computing graph features for 1000000 pairs...



Graph Score Stats:
       graph_entailment_score  graph_equivalence_score
count               1000000.0                1000000.0
mean                      0.0                      0.0
std                       0.0                      0.0
min                       0.0                      0.0
25%                       0.0                      0.0
50%                       0.0                      0.0
75%                       0.0                      0.0
max                       0.0                      0.0


In [15]:
df_labeled = fea.add_graph_features(
    df=df_labeled,
    entailment_df=df_obs_ent,
    id1_col="id1",
    id2_col="id2",
    verdict_col="verdict",
    positive_label="YES",
    decay=0.9,  # Confidence drops by 10% per extra hop
    max_hops=5
)

print("\nGraph Score Stats:")
print(df_candidates[['graph_entailment_score', 'graph_equivalence_score']].describe())

Building Directed Entailment Graph...
Computing graph features for 10 pairs...

Graph Score Stats:


       graph_entailment_score  graph_equivalence_score
count               1000000.0                1000000.0
mean                      0.0                      0.0
std                       0.0                      0.0
min                       0.0                      0.0
25%                       0.0                      0.0
50%                       0.0                      0.0
75%                       0.0                      0.0
max                       0.0                      0.0


In [16]:
df_candidates = df_candidates.dropna(subset=['cos_sim_neighbor_score'])
df_labeled = df_labeled.dropna(subset=['cos_sim_neighbor_score'])

print(f"After dropna: {len(df_candidates)} candidates, {len(df_labeled)} labeled")
if len(df_labeled) == 0:
    print("WARNING: No labeled rows with valid scores — model training will be skipped upstream.")

After dropna: 1000000 candidates, 10 labeled


# Predicting Entailment (can change model pipeline to something other than logistic)

In [17]:
# Feature Engineering & Model Training
# Updated features list to include robust graph metrics
features = [
    'cos_sim_neighbor_score', 
    #'nli_neighbor_score', 
    #'graph_entailment_score', 
    #'graph_equivalence_score'
]
target = 'verdict'
positive_label = 'YES'

# Generate BERT results for df_labeled


print(f"Training dataset: {len(df_labeled)} pairs with features and verdicts")
print(f"Prediction dataset: {len(df_candidates)} pairs with features (no verdicts)")

# 2. (Optional) Run Hyperparameter Optimization with Optuna
try:
    print("\n>>> Optimizing Boosting Hyperparameters with Optuna...")
    # This might take a minute but will find scientifically best parameters
    best_params = fea.optimize_boosting_hyperparameters(
        df=df_labeled,  # TRAIN ON LABELED DATA!
        feature_cols=features,
        target_col=target,
        positive_label=positive_label,
        n_trials=30 
    )
    
    # Add monotonic constraint assumption back if we believe in it
    # (Optuna doesn't optimize this structure, it optimizes numbers)
    if best_params:
        best_params['enforce_monotonicity'] = True 
        
except Exception as e:
    print(f"\nOptimization skipped or failed: {e}")
    print("Using conservative defaults.")
    best_params = {
        'learning_rate': 0.05,
        'max_iter': 300,
        'enforce_monotonicity': True
    }


# 3. Run Comparative Analysis using helper function
# Updated: Now uses Cross-Validation internally to prevent Overfitting!
comparison_df, best_model_name = fea.compare_entailment_models(
    df=df_labeled,  # TRAIN ON LABELED DATA!
    feature_cols=features,
    target_col=target,
    model_names=["logistic", "spline", "tree", "boosting"], 
    positive_label=positive_label,
    **best_params # Unpack the best parameters here
)

# --- Display Results ---
print("\nComparison Results (Sorted by ROC-AUC):")
display(comparison_df)

print(f"\n>>> Selected '{best_model_name}' model for downstream processing.")

Training dataset: 10 pairs with features and verdicts
Prediction dataset: 1000000 pairs with features (no verdicts)

>>> Optimizing Boosting Hyperparameters with Optuna...
Optuna not installed. Please run: pip install optuna
Running comparative analysis on 10 samples...
Features: ['cos_sim_neighbor_score']

--- Training logistic ---
Training Logistic Regression...


Model (logistic) Train Accuracy: 1.0000


--- Training spline ---
Training Spline Logistic Regression...
Model (spline) Train Accuracy: 1.0000


--- Training tree ---
Training Decision Tree Classifier...
Model (tree) Train Accuracy: 1.0000
--- Training boosting ---
Training Histogram Gradient Boosting Classifier (lr=0.05, iter=200)...


Model (boosting) Train Accuracy: 0.8000

Comparison Results (Sorted by ROC-AUC):


,ROC-AUC (CV),Log Loss,Separation,Mean Prob (YES),Mean Prob (NO)
Model,,,,,
logistic,1.0,3.112647e-01,0.559907,0.852464,0.292556
spline,1.0,4.928950e-01,0.224716,0.614823,0.390106
tree,1.0,2.220446e-16,1.000000,1.000000,0.000000
boosting,0.5,6.931472e-01,0.000000,0.500000,0.500000



>>> Selected 'logistic' model for downstream processing.


In [18]:
print(f"Retraining '{best_model_name}' with optimized parameters on labeled data...")

# Train the model on df_labeled_with_features (which has verdicts)
best_pipeline = fea.train_entailment_model(
    df=df_labeled,  # TRAIN ON LABELED DATA!
    feature_cols=features,
    target_col=target,
    method=best_model_name,
    positive_label=positive_label,
    **best_params
)


df_candidates = fea.predict_entailment_probabilities(
    df_candidates,
    model_pipeline=best_pipeline,
    feature_cols=features,
    new_col='entailment_probability'
)

print(f"\nPrediction stats:")
print(f"  Min probability: {df_candidates['entailment_probability'].min():.4f}")
print(f"  Max probability: {df_candidates['entailment_probability'].max():.4f}")
print(f"  Mean probability: {df_candidates['entailment_probability'].mean():.4f}")

df_labeled.head()

Retraining 'logistic' with optimized parameters on labeled data...
Training Logistic Regression...
Model (logistic) Train Accuracy: 1.0000



Prediction stats:
  Min probability: 0.0000
  Max probability: 0.9956
  Mean probability: 0.0042


c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,text1,text2,verdict,equivalents1,equivalents2,alpha,new_cos_sim_score,cos_sim_neighbor_score,graph_entailment_score,graph_equivalence_score
0,B0448006p,B1089003p,The emphasis on a singular sovereign power in ...,The accountability of the sovereign to God emp...,NO,[],[],NaN,0.763184,0.763184,0.0,0.0
1,B1089003p,B0448006p,The accountability of the sovereign to God emp...,The emphasis on a singular sovereign power in ...,NO,[],[],NaN,0.763184,0.763184,0.0,0.0
2,B0674004p,S0051696006p,Maintaining order and justice is essential for...,Maintaining democratic oversight is crucial to...,NO,[],[],NaN,0.738770,0.738770,0.0,0.0
3,S0051696006p,B0674004p,Maintaining democratic oversight is crucial to...,Maintaining order and justice is essential for...,NO,[],[],NaN,0.738770,0.738770,0.0,0.0
4,B0427001sc,B0596001sc,Popular Estates are essential for a just monar...,Lawful political authority is essential for so...,NO,[],[],NaN,0.765625,0.765625,0.0,0.0


# Optimize threshold 

In [19]:
# Predict on labeled data to find optimal thresholds
print("Predicting on labeled data for threshold optimization...")
df_labeled_with_features = fea.predict_entailment_probabilities(
    df_labeled,
    model_pipeline=best_pipeline,
    feature_cols=features,
    new_col='entailment_probability'
)

# We use the generic 'entailment_probability' column which now holds the best model's output
importlib.reload(fea)

results = fea.find_best_thresholds(
    df=df_labeled_with_features,  # USE LABELED DATA FOR THRESHOLD TUNING!
    score_col="entailment_probability",
    verdict_col="verdict",
    positive_label="YES"
)

df_labeled_with_features.head()

Predicting on labeled data for threshold optimization...


c:\Users\aesteva\Downloads\python-3.13.12-embed-amd64\Lib\site-packages\pandas\io\formats\format.py:1466: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id1,id2,text1,text2,verdict,equivalents1,equivalents2,alpha,new_cos_sim_score,cos_sim_neighbor_score,graph_entailment_score,graph_equivalence_score,entailment_probability
0,B0448006p,B1089003p,The emphasis on a singular sovereign power in ...,The accountability of the sovereign to God emp...,NO,[],[],NaN,0.763184,0.763184,0.0,0.0,0.249822
1,B1089003p,B0448006p,The accountability of the sovereign to God emp...,The emphasis on a singular sovereign power in ...,NO,[],[],NaN,0.763184,0.763184,0.0,0.0,0.249822
2,B0674004p,S0051696006p,Maintaining order and justice is essential for...,Maintaining democratic oversight is crucial to...,NO,[],[],NaN,0.738770,0.738770,0.0,0.0,0.144043
3,S0051696006p,B0674004p,Maintaining democratic oversight is crucial to...,Maintaining order and justice is essential for...,NO,[],[],NaN,0.738770,0.738770,0.0,0.0,0.144043
4,B0427001sc,B0596001sc,Popular Estates are essential for a just monar...,Lawful political authority is essential for so...,NO,[],[],NaN,0.765625,0.765625,0.0,0.0,0.262831


In [20]:
print("Best tau (accuracy):", results["best_tau_accuracy"],
      "Accuracy:", results["best_accuracy"])

print("Best tau (F1):", results["best_tau_f1"],
      "F1:", results["best_f1"])

print("Best tau (TP):", results["best_tau_tp"],
      "TP:", results["max_true_positives"])

print("Best tau (precision):", results["best_tau_precision"],
      "prec:", results["best_precision"])

print("Best tau (recall):",    results["best_tau_recall"],
      "rec:",  results["best_recall"])

Best tau (accuracy): 0.26283118426905855 Accuracy: 1.0
Best tau (F1): 0.26283118426905855 F1: 1.0
Best tau (TP): 0.1440428044139397 TP: 2
Best tau (precision): 0.26283118426905855 prec: 1.0
Best tau (recall): 0.1440428044139397 rec: 1.0


In [21]:
results["best_taus_table"]

,tau,TP,TN,FP,FN,accuracy,precision,recall,f1
0,0.144043,2,2,6,0,0.4,0.25,1.0,0.4
1,0.262831,2,8,0,0,1.0,1.00,1.0,1.0


In [22]:
import importlib
import plotly.io as pio
import free_entailments_algorithm_utils as fea
importlib.reload(fea) 

# Ensure Plotly renders appropriately for notebook/vscode context
pio.renderers.default = "notebook_connected" 

# This also calculates 'best_tau_low_send' (Top 1-5% Candidates)
# Use labeled data (with verdicts) for threshold finding!

results = fea.find_best_thresholds(
    df=df_labeled_with_features,  # FIX: Use labeled data, not candidates!
    score_col="entailment_probability", 
    verdict_col="verdict",
    positive_label="YES"
)


tau_low_send = results.get('best_tau_low_send', 0.95)

print("\nLow-Send Optimization (Candidate Selection):")
print(f"Selected Low-Send Threshold: {tau_low_send:.6f}")
if "low_send_table" in results:
    display(results["low_send_table"])

print("\n>>> Interactive Analysis: LLM Savings vs Threshold")

markers_to_show = {
    "Optimization (Top %)":    tau_low_send,
    "Max Accuracy":            results["best_tau_accuracy"],
    "Max F1":                  results["best_tau_f1"]
}


# Sent = Prob > Threshold
fig = fea.plot_llm_savings_over_thresholds(
    df=df_labeled_with_features,  # FIX: Use labeled data for plotting too!
    prob_col="entailment_probability",
    verdict_col="verdict",
    positive_label="YES",
    step=0.01,
    markers=markers_to_show
)
fig.show()


Low-Send Optimization (Candidate Selection):
Selected Low-Send Threshold: 0.903024


,target_percentile,tau,sent_rate,FN,TP,FP,TN
0,0.01,0.903024,0.0,2,0,0,8
1,0.02,0.903024,0.0,2,0,0,8
2,0.03,0.903024,0.0,2,0,0,8
3,0.04,0.903024,0.0,2,0,0,8
4,0.05,0.903024,0.0,2,0,0,8



>>> Interactive Analysis: LLM Savings vs Threshold


In [23]:
import importlib
import free_entailments_algorithm_utils as fea
importlib.reload(fea)

print("--- Defining Threshold for LLM ---")

# Strategy: Send everything above a certain confidence threshold.
# We use the 'Minimize False Negatives' strategy (Cost Sensitive) to find a threshold
# effectively filtering out 'Definite Negatives' while keeping all potential Candidates.
# Cost Ratio 1:5 means we punish missing a Yes (FN) 5x more than sending a useless No (FP).

tau = fea.get_optimal_threshold_minimize_fn(strategy='cost', cost_fn=5.0)

print(f"Selected Threshold: {tau:.4f} (Send if Score > {tau:.4f})")
print(f"Logic: Minimize FN (Don't miss Entailments). Auto-Reject scores <= {tau:.4f}.")

# # 2. Estimate Cost
# print("\n--- Cost Analysis ---")
# cost = fea.estimate_deepseek_cost(
#     df=df_candidates, 
#     prob_col='entailment_probability', 
#     threshold=tau,
#     model="deepseek-reasoner"
# )

# 3. Generate Final DataFrame for the LLM
print("\n--- Generating File ---")
df_final = fea.generate_final_df(
    df=df_candidates, 
    prob_col='entailment_probability', 
    threshold=tau,
    df_clause=df_clause,
    id_col='sentence_id',
    text_col='sentence'
)
# df_final.to_csv("llm_batch_final.csv", index=False)

--- Defining Threshold for LLM ---
Selected Threshold: 0.1667 (Send if Score > 0.1667)
Logic: Minimize FN (Don't miss Entailments). Auto-Reject scores <= 0.1667.

--- Generating File ---


--- Generating LLM Batch ---
Original Count: 1,000,000
Filtered Count: 2,663 (0.3%)
Condition:      P > 0.1667 (Send High Confidence Pairs)


In [24]:
# Record outputs for papermill using scrapbook
import scrapbook as sb

# Save df_final and cost for this iteration
sb.glue('df_final', df_final)

# Convert Plotly figure to HTML for serialization
fig_html = fig.to_html(include_plotlyjs='cdn')
sb.glue('fig_html', fig_html)

print("\n✓ Outputs recorded for papermill retrieval")


✓ Outputs recorded for papermill retrieval
